<strong>주가 예측 프로젝트 prac 3</strong>

- sql 라이브러리 사용 <br>
- 다음날 종가 변동률 분포 확인
- 변수명 변경

<strong> 수행시간</strong> 
- 과제 1: 0초
- 과제 2: 1분
- 과제 3: 2초

리스트와 비교해보면 약 3분, 딕셔너리와 비교해보면 약 2분이 차이남

In [1]:
import os
import time
import pandas as pd
import numpy as np
import datetime
import pymysql
from sqlalchemy import create_engine
import FinanceDataReader as fdr
from tqdm import tqdm
import pickle
import warnings

In [2]:
warnings.filterwarnings('ignore')

# SQL
- tab separated 형식
- fdr 라이브러리를 사용하여 전체 종목 코드 다운받기
- 상장일 2018년 1월 2일 이후 종목 텍스트 파일로 저장하기<br>

In [3]:
stock = pd.read_csv('../data/raw_data.csv',index_col = 0)
stock['종목코드'] = stock['종목코드'].apply(lambda x : str(x).zfill(6))
stock.head()

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,ES큐브,050120,생활용품 도매업,"텐트, 원 ·부자재",2002-04-25,12월,신희민,http://www.escube.co.kr,서울특별시
1,ISC,095340,전자부품 제조업,반도체테스트소켓,2007-10-01,12월,"김정렬, 김상욱 (각자 대표이사)",http://isc21.kr,경기도
2,JW신약,067290,의약품 제조업,"매니큐어세트, 손톱깎이",2003-02-11,12월,백승호,http://www.cwsy.co.kr,서울특별시
3,KB오토시스,024120,자동차 신품 부품 제조업,브레이크패드,1994-12-07,12월,김신완,http://www.kbautosys.com,충청남도
4,KH바텍,060720,전자부품 제조업,"휴대폰부품(SHIELD, BRACKET)",2002-05-21,12월,남광희,http://www.khvatec.com,경상북도


In [4]:
result = {}
lst_stock = stock.values.tolist()

OF = open('../data/assignment1.txt','w')
for row in tqdm(lst_stock):
    code, name,date = row[1],row[0],row[4]
    if date <= '2018-01-02':
        result[code] = date
        OF.write(f'{code}\t{name}\n')

100%|██████████████████████████████████████████████████████████████████████████| 1910/1910 [00:00<00:00, 328850.24it/s]


cursor : 행 단위 작업을 효율적으로 하기 위한 방식으로 테이블에서 여러 개의 행을 조회한 후 </br>
쿼리의 결과를 한행씩 처리하는 방식 </br>
SELECT한 결과를 반복 작업해줘야 할 경우 유용하게 사용 가능함

In [13]:
#con = pymysql.connect()
#cursor = con.cursor()

In [6]:
stock.head()

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,ES큐브,050120,생활용품 도매업,"텐트, 원 ·부자재",2002-04-25,12월,신희민,http://www.escube.co.kr,서울특별시
1,ISC,095340,전자부품 제조업,반도체테스트소켓,2007-10-01,12월,"김정렬, 김상욱 (각자 대표이사)",http://isc21.kr,경기도
2,JW신약,067290,의약품 제조업,"매니큐어세트, 손톱깎이",2003-02-11,12월,백승호,http://www.cwsy.co.kr,서울특별시
3,KB오토시스,024120,자동차 신품 부품 제조업,브레이크패드,1994-12-07,12월,김신완,http://www.kbautosys.com,충청남도
4,KH바텍,060720,전자부품 제조업,"휴대폰부품(SHIELD, BRACKET)",2002-05-21,12월,남광희,http://www.khvatec.com,경상북도


 선정한 종목 중에서 거래대금 1조 이상인 날짜 수집
- Date, Open, High, Close, Volume, Change

In [7]:
dic_code2date = {}

for code in tqdm(result.keys()):
    sql_query = '''
            SELECT * FROM 
            stock_{}
            WHERE Date
            BETWEEN '2018-01-01' AND '2021-12-31'
    '''.format(code)
    
    stock = pd.read_sql(sql = sql_query , con = con)
    lst_stock = stock.values.tolist()
    for row in lst_stock:
        date, close , volume, trading_val = row[0].date().strftime('%Y%m%d'), row[4],row[5],row[4]*row[5]
        
        if trading_val >= 1000000000000:
            if code not in dic_code2date:
                dic_code2date[code] = []
                dic_code2date[code].append(date)
            else:
                dic_code2date[code].append(date)

100%|██████████████████████████████████████████████████████████████████████████████| 1910/1910 [03:03<00:00, 10.41it/s]


In [8]:
OF = open('../data/assignment2.txt', 'w')
for code,date in dic_code2date.items():
    OF.write(f"{code}\t{date}\n")
OF.close()

In [16]:
OF=open('../data/assignment3.txt','w')

lst_result = []

for code in tqdm(dic_code2date.keys()):
    sql_query = '''
                SELECT *
                FROM stock_{}
                WHERE Date
                BETWEEN '2018-01-01' AND '2021-12-31'
                '''.format(code)
    stock = pd.read_sql(sql = sql_query, con = con)
    lst_stock = stock.values.tolist()  
    
    for i, row in enumerate(lst_stock):
        
        if (i < 10) or (i >= len(lst_stock)-1):
            continue
        
   
        date = row[0].date().strftime('%Y%m%d')
        if date not in dic_code2date[code]:
            
            continue
        D9_date = lst_stock[i-10:i+1]
        lst_trading = []
        
        for row_trading in D9_date:
            Open, High, Low, Close, Volume = row_trading[1:6]
            Trading = Close * Volume
            lst_trading += [Open, High, Low, Close, Trading]
            
        data = ','.join(map(str,lst_trading))
        
        target = int(lst_stock[i+1][-1]>=0.02)
        result = '{}\t{}\t{}\t{}\n'.format(code, date, data, target)
        lst_result.append([code, date, Trading, target])
        OF.write(result)
OF.close()

100%|██████████████████████████████████████████████████████████████████████████████████| 82/82 [00:08<00:00,  9.73it/s]
